In [1]:
import numpy as np
import networkx as nx
import random
import json

## Parameters

In [2]:
p_new_node = 0.5
community_mat = np.array([[0.9, 0.1], [0.1, 0.9]])
N_timesteps = 6
N_nodes = 50
N_edge_per_time = 20

In [6]:
community_mat

array([[0.9, 0.1],
       [0.1, 0.9]])

In [7]:
community_mat = np.array([[0.9, 0.1, 0.1, 0.1], [0.1, 0.9, 0.1, 0.1], [0.1, 0.1, 0.9, 0.1], [0.1, 0.1, 0.1, 0.9]])

In [8]:
len(community_mat)

4

## Generate

In [38]:
def normalize_probs(p):
    p = np.asarray(p).astype('float64')
    p = p / np.sum(p)
    return p

In [39]:
def generate_hypergraph(p_new_node, community_mat, N_timesteps, N_edge_per_time):
    G = nx.Graph()
    counter = 0
    node_to_com = {}
    
    all_nodes = range(N_nodes)
    
    for node in all_nodes:
        name = ''.join(random.choice([chr(i) for i in range(ord('a'),ord('z'))]) for _ in range(6))
        node_to_com[node] = np.random.randint(len(community_mat), size=1)[0]
        G.add_node(node, label="person", name=name, comm=int(node_to_com[node]))
    
    for time in range(1, N_timesteps+1):
        for new_edge in range(N_edge_per_time):
            node_id = np.random.randint(N_nodes, size=1)[0]
            nodes_of_edge = [node_id]
            
            r = 0
            while r < p_new_node:    
                node_props = [community_mat[node_to_com[node_id], node_to_com[node]] for node in all_nodes]
                new_node = np.random.choice(all_nodes, p=normalize_probs(node_props))
                nodes_of_edge.append(new_node)
                r = random.random()
                
            hyperedge = "h" + str(counter)
            counter += 1
            G.add_node(hyperedge, time=time, label="document")
            for node in nodes_of_edge:
                G.add_edge(hyperedge, node, label="edge")
                
    return G  

In [161]:
G = generate_hypergraph(p_new_node, community_mat, N_timesteps, N_edge_per_time)

1
2
3
4
5
6


## Export

In [162]:
graph_json = nx.node_link_data(G)

In [163]:
graph_json["metadata"] = {
            "datasetName": "test",
            "edgeType": "label",
            "entityType": "label",
            "name": "name",
            "source_entity_type": "document",
            "target_entity_type": "person",
            "time_key": "time",
            # "entityTypes": [entity_type for entity_type in self.node_labels],
            # "attributes": [{"name": property.name, "type": property.type} for property in self.properties]
        }

In [164]:
fp = "test_gen.json"

In [165]:
with open(fp, "w+") as path:
    json.dump(graph_json, path)

## Agent based Network model

In [40]:
# Sample list of animal names
animal_names = {
    "cat", "dog", "elephant", "giraffe", "lion", "tiger", "panda", "koala", "kangaroo",
    "zebra", "hippo", "bear", "wolf", "fox", "rhino", "cheetah", "penguin", "dolphin",
    "whale", "shark", "eagle", "hawk", "falcon", "owl", "sparrow", "hummingbird", "parrot",
    "peacock", "duck", "swan", "goose", "rabbit", "squirrel", "deer", "moose", "gazelle",
    "monkey", "gorilla", "chimpanzee", "orangutan", "lemur", "hyena", "leopard", "panther",
    "crocodile", "alligator", "snake", "turtle", "frog", "lizard", "octopus", "squid", "crab",
    "lobster", "shrimp", "starfish", "jellyfish", "seahorse", "snail", "butterfly", "bee",
    "ant", "beetle", "ladybug", "spider", "scorpion", "tarantula", "hedgehog", "armadillo",
    "platypus", "kookaburra", "emu", "wombat", "tasmanian devil", "kiwi", "ostrich", "rhinoceros beetle",
    "buffalo", "moose", "raccoon", "skunk", "porcupine", "hedgehog", "polar bear", "walrus",
    "seal", "otters", "sloth", "anteater", "aardvark", "reindeer", "antelope", "caribou",
    "yak", "gnu", "hippopotamus", "hyena", "jackal", "elephant seal", "meerkat", "armadillo",
    "black bear", "chinchilla", "fennec fox", "lynx", "jaguar", "quokka", "tarsier", "anteater",
    "capybara", "okapi", "lemur", "leopard", "manatee", "numbat", "pangolin", "red panda",
    "serval", "tapir", "bongo", "bongo", "fossa", "gibbon", "saki monkey", "tit", "toucan",
    "star-nosed mole", "springhare", "armadillo girdled lizard", "numbat", "sugar glider", "wallaroo",
    "red-handed tamarin", "green basilisk", "glass frog", "flying dragon", "pufferfish", "blowfish",
    "lionfish", "scorpionfish", "mandarin fish", "blue tang", "parrotfish", "yellow tang",
    "flounder", "swordfish", "goblin shark", "hammerhead shark", "bull shark", "mako shark",
    "giant manta ray", "stingray", "sea turtle", "leatherback turtle", "box turtle",
    "green sea turtle", "giant panda", "red fox", "sea otter", "jaguar", "siberian tiger",
    "bald eagle", "harpy eagle", "humpback whale", "blue whale", "orca", "bottlenose dolphin",
    "gray wolf", "golden eagle", "african elephant", "indian elephant", "saltwater crocodile",
    "american alligator", "black rhinoceros", "white rhinoceros", "black mamba", "king cobra",
    "reticulated python", "green anaconda", "nile crocodile", "poison dart frog",
    "giant african millipede", "giant centipede", "tarantula", "giant desert hairy scorpion",
    "black widow spider", "african lion", "cheetah", "leopard", "giraffe", "african elephant",
    "grizzly bear", "polar bear", "red kangaroo", "blue whale", "bengal tiger", "siberian tiger",
    "american bison", "american flamingo", "african penguin", "african grey parrot",
    "green tree python", "chimpanzee", "koala", "arctic fox", "snow leopard", "red panda",
    "sloth", "raccoon", "meerkat", "black-footed ferret", "beaver", "giant panda", "honey badger",
    "prairie dog", "sea lion", "walrus", "gray whale", "elephant seal", "macaw", "scarlet macaw",
    "cockatoo", "hummingbird", "woodpecker", "kingfisher", "puffin", "hornbill", "horned owl",
    "barn owl", "peregrine falcon", "osprey", "buzzard", "red-tailed hawk", "harpy eagle",
    "bald eagle", "bearded vulture", "black vulture", "turkey vulture", "pheasant", "quail",
    "partridge", "guinea fowl", "dove", "pigeon", "stork", "crane", "flamingo", "pelican",
    "swan", "goose", "duck", "heron", "ibis", "egret", "spoonbill", "albatross", "seagull",
    "tern", "frigatebird", "penguin", "ostrich", "emu", "kiwi", "cassowary", "rhea", "toad",
    "frog", "salamander", "newt", "caecilian", "crocodile", "alligator", "gavial", "iguana",
    "chameleon", "komodo dragon", "gecko", "monitor lizard", "skink", "cobra", "viper",
    "rattlesnake", "black mamba", "king cobra", "anaconda", "python", "boa constrictor",
    "sea snake", "grass snake", "water snake", "garter snake", "hognose snake", "milk snake"}

def name_generator():
    random_animal = random.sample(animal_names, 1)[0]
    # animal_names.remove(random_animal)
    return random_animal
    # return ''.join(random.choice([chr(i) for i in range(ord('a'),ord('z'))]) for _ in range(6))

In [41]:
name_generator()

/tmp/ipykernel_284591/2158273777.py:47: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random_animal = random.sample(animal_names, 1)[0]


'pigeon'

In [42]:
class Node():
    def __init__(self, id, name, com):
        self.id = id
        self.name = name
        self.com = com
        self.dead = False
        
    def timestep_run(self, N_coms, p_dying = 0.01, p_switch_com=0.08):
        self.N_coms = N_coms
        r = random.random()
        r2 = random.random()
        if r < p_dying:
            self.dead = True
        if r2 < p_switch_com:
            self.switch_comm()
            
    def switch_comm(self):
        new_com = -1
        while self.com != new_com:
            new_com = random.randint(0, self.N_coms - 1)
        self.com = int(new_com)

In [43]:
com_counter = 0

class Community:
    def __init__(self):
        self.id = com_counter
        com_counter += 1

In [44]:
a = np.full([3, 3], 3.14)
a.flat[0::4] = -2 

In [45]:
a

array([[-2.  ,  3.14,  3.14],
       [ 3.14, -2.  ,  3.14],
       [ 3.14,  3.14, -2.  ]])

In [72]:
class Generator:
    def __init__(self):
        self.N_timesteps = 8
        self.N_nodes = 60
        # self.N_edge_per_time = int(self.N_nodes / 6)
        self.N_edge_per_time = 25
        
        self.p_new_node = 0.05
        self.p_add_he_node = 0.3
        
        self.p_com_dying = 0.1
        self.p_com_appear = 0.2
        self.p_com_merge = 0.15
        
    
        self.counter = 0
        self.N_coms = 5
        
        # self.community_mat = np.array([[1.5, 0.1, 0.1], [0.1, 1.5, 0.1], [0.1, 1.5, 2]])
        # self.community_mat = np.array([[1.5, 0.1], [0.1, 1.5]])
        # self.community_mat = np.array([[2, 0.1, 0.1, 0.1], [0.1, 2, 0.1, 0.1], [0.1, 0.1, 2, 0.1], [0.1, 0.1, 0.1, 2]])
        # self.community_mat = np.full([self.N_coms, self.N_coms], 1)
        # self.community_mat.flat[0::(self.N_coms + 1)] = 40
        
        self.community_mat = np.full([self.N_coms, self.N_coms], 0.05)
        self.community_mat.flat[0::(self.N_coms + 1)] = 0.5
        
        print(self.community_mat)
        
        self.coms_dead = set()
    
        
    def nodes_of_community(self, com_id):
        return [node for node in self.all_nodes if node.com == com_id]
        
    def start(self):
        self.G = nx.Graph()
        # self.node_to_com = {}
        self.all_nodes = []
        for idnode in range(self.N_nodes):
            name = self.new_name()
            com = int(np.random.randint(len(self.community_mat), size=1)[0])
            node = Node(idnode, name, com)
            self.all_nodes.append(node)
            
#             TODO : com attribute
            self.G.add_node(node.id, label="person", name=name, comm=node.com)
        
    def living_nodes(self):
        return [node for node in self.all_nodes if not node.dead]
    
    def living_nodes_ids(self):
        return [node.id for node in self.living_nodes()]
    
    def timestep_run(self):
        for node in self.living_nodes():
            node.timestep_run(len(self.community_mat))
            
        for com_id, community in enumerate(self.community_mat):
            r = random.random()
            if com_id not in self.coms_dead and r < self.p_com_dying:
                print("COM DEAD")
                self.coms_dead.add(com_id)
                nodes = self.nodes_of_community(com_id)
                for node in nodes:
                    node.dead = True
                    
        r = random.random()
        if r < self.p_com_appear:
            print("NEW COM")
            self.N_coms += 1
            self.community_mat = np.full([self.N_coms, self.N_coms], 1)
            self.community_mat.flat[0::(self.N_coms + 1)] = 40
            
            n_new_nodes = int(len(self.living_nodes()) / self.N_living_coms())
            for i in range(n_new_nodes):
                self.add_node(self.N_coms - 1)
                
                
    def add_node(self, com):
        node_id = len(self.all_nodes)
        name = self.new_name()

        node = Node(node_id, name, com)
        self.all_nodes.append(node)
        self.G.add_node(node_id, label="person", name=name, comm=com)
        
    def new_name(self):
        name = name_generator()
        if name in dict(self.G.nodes(data="name")).values():
            name = name + "'"
        return name
            
    def N_living_coms(self):
        return self.N_coms - len(self.coms_dead)
        
    def run(self):
        self.start()
        for time in range(1, self.N_timesteps+1):
            self.generate_hyperedges(time)
            self.timestep_run()
            
    def generate_hyperedges(self, time):
        self.generate_hyperedges_static(time)
        
    def generate_hyperedges_blockmodel(self, time):
        for i, node in enumerate(self.living_nodes()):
            for node2 in self.living_nodes()[(i+1):]:
                if node != node2:
                    r = random.random()
                    if r < self.community_mat[node.com, node2.com]:
                        nodes_of_edge = [node.id, node2.id]
                        r2 = random.random()
                        while r2 < self.p_add_he_node:    
                            new_node = self.select_node(node, True)
                            nodes_of_edge.append(new_node.id)
                            r2 = random.random()
                        
                        hyperedge = "h" + str(self.counter)
                        self.counter += 1
                        self.G.add_node(hyperedge, ts=time, label="document")
                        for node in nodes_of_edge:
                            self.G.add_edge(hyperedge, node, label="edge")
        
    def generate_hyperedges_static(self, time):
        for new_edge in range(self.N_edge_per_time):
            first_new_node = self.select_node(False)
            nodes_of_edge = [first_new_node.id]

            r = 0
            while r < self.p_add_he_node:    
                new_node = self.select_node(first_new_node, True)
                nodes_of_edge.append(new_node.id)
                r = random.random()

            hyperedge = "h" + str(self.counter)
            self.counter += 1

            # self.G.add_node(hyperedge, time=time, label="document")
#                 Paohvis format
            self.G.add_node(hyperedge, ts=time, label="document")

            for node in nodes_of_edge:
                self.G.add_edge(hyperedge, node, label="edge")
                    
    def select_node(self, neighbor=None, weights=False):
        r0 = random.random()
        if r0 < self.p_new_node:
            # New Node
            node_id = len(self.all_nodes)
            name = self.new_name()
            
            # new_com = random.randint(0, len(self.community_mat) - 1)
            new_com = random.choice(list(set(range(self.N_coms)) - self.coms_dead))
            
            node = Node(node_id, name, new_com)
            self.all_nodes.append(node)
            self.G.add_node(node_id, label="person", name=name, comm=new_com)
        else:
            if weights:
#                 TODO: Normalize given the number of nodes
                node_props = [self.community_mat[neighbor.com, node.com] for node in self.living_nodes()]
                # print(normalize_probs(node_props))
                node = np.random.choice(self.living_nodes(), p=normalize_probs(node_props))
            else:
                node = np.random.choice(self.living_nodes(), size=1)[0]
        return node
    
    def to_json(self):
        graph_json = nx.node_link_data(self.G)        
        # graph_json["metadata"] = {
        #     "datasetName": "test",
        #     "edgeType": "label",
        #     "entityType": "label",
        #     "name": "name",
        #     "source_entity_type": "document",
        #     "target_entity_type": "person",
        #     "time_key": "time",
        #     # "entityTypes": [entity_type for entity_type in self.node_labels],
        #     # "attributes": [{"name": property.name, "type": property.type} for property in self.properties]
        # }
        
#         For Paohvis
        graph_json["metadata"] = {
            "datasetName": "test",
            "edgeType": "label",
            "entityType": "label",
            "name": "name",
            "source_entity_type": "document",
            "target_entity_type": "person",
            "time_key": "time",
            "format": "2.1.0",
            "entity_type": "label",
        }
        
        return graph_json
        
    def export(self):
        fp = f"{self.N_timesteps}times_{self.N_coms}coms.json"
        with open(fp, "w+") as path:
            json.dump(self.to_json(), path)
            
        with open("output.json", "w+") as path:
            json.dump(self.to_json(), path)
        

In [73]:
g = Generator()

[[0.5  0.05 0.05 0.05 0.05]
 [0.05 0.5  0.05 0.05 0.05]
 [0.05 0.05 0.5  0.05 0.05]
 [0.05 0.05 0.05 0.5  0.05]
 [0.05 0.05 0.05 0.05 0.5 ]]


In [74]:
g.run()
g.export()

NEW COM
COM DEAD
COM DEAD
COM DEAD
NEW COM
COM DEAD
COM DEAD


/tmp/ipykernel_284591/2158273777.py:47: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random_animal = random.sample(animal_names, 1)[0]
